In [ ]:
import os
import random
import numpy as np


import tensorflow as tf
import warnings
import sys



# Set environment variables for deterministic GPU operations
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

# Set random seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Complete warning suppression
warnings.simplefilter("ignore")
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings too

# Redirect stderr temporarily to suppress low-level warnings
class SuppressOutput:
    def __enter__(self):
        self._original_stderr = sys.stderr
        sys.stderr = open(os.devnull, 'w')
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stderr.close()
        sys.stderr = self._original_stderr



# GPU setup - do this BEFORE any other TensorFlow operations
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")

import flopy
import pyemu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import herebedragons as hbd
import shutil
from pyemu.emulators import DSIAE


print("Python version:", sys.version)
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())

# This should show your GPU
print("Physical devices:", tf.config.list_physical_devices())

# Intro

In a previous notebook we ran good 'ole vanilla PCA based DSI. Here we introduce the new DSI-AutoEncoder class.

In standard PCA-based data space inversion, the relationship between model parameters and observations is represented in a reduced-dimensional space defined by linear combinations of principal components. This linear reduction efficiently captures dominant variance but may miss nonlinear features of the system. In contrast, autoencoder-based data space inversion uses a neural network to learn a nonlinear mapping between the high-dimensional model outputs and a compact latent representation. This allows the autoencoder to capture more complex, nonlinear relationships in the data, potentially improving inversion performance in systems where responses are not well described by linear structures.

### Autoencoders

Autoencoders are neural networks designed to learn efficient, compressed representations of data. They consist of two parts: an **encoder**, which maps the input data into a lower-dimensional latent space, and a **decoder**, which reconstructs the original data from this latent representation. By training the network to minimize the difference between the input and reconstructed output, autoencoders learn to capture the most important, often nonlinear, features of the data.

Autoencoders can leverage specialized architectures like **convolutional** and **LSTM** layers to better capture structure in the data. **Convolutional autoencoders** are well suited for spatially structured data, as they learn local spatial features and patterns. **LSTM-based autoencoders**, on the other hand, are designed for sequential or time-series data, capturing temporal dependencies and dynamics in the latent representation. By using these architectures, autoencoders can more effectively encode and reconstruct complex spatial or temporal relationships in the data.

## what we are going to do

In this noteook we are going to throught the mechanics of using the `pyemu.DSIAE` class and some of the caveats. This is a vanilla AE (think non-linear PCA), without any fancy LSTM or convolutional layers.

THe first section mirrors the same steps as in the DSI notebook.


# Getting ready
## Load the Prior MC results

In [ ]:
# specify the temporary working folder
org_md = os.path.join('master_prior_mc')
pst = pyemu.Pst(os.path.join(org_md,'pest.pst'))
oe = pst.ies.obsen.copy()
oe.head()

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws='model', load_only=[],verbosity_level=0)
gwf = sim.get_model('gwf')

## Choose the truth

Lets choose one of the realizations as the rtuth, then remove if form the training data. We are going to select an inconvenient truth: one in which the max temperature plume extends a bit farther than most.

In [ ]:
obs = pst.observation_data
obsnmes = obs.loc[obs.oname=="temp"].obsnme.tolist()

# find columns in data[obsnmes] where 50% of the values are below 16.01
cols = oe[obsnmes].columns[(oe[obsnmes] <= 16.005).mean() > 0.5]
print(len(cols), len(obsnmes))

# find col in cols with highest std
stds = oe[cols].std()
cols = stds.sort_values(ascending=False).index.tolist()


target_col = cols[0]
print(target_col)

fig,ax = plt.subplots(1,1,figsize=(4,4))

oe.loc[:,target_col].hist(ax=ax)

truth_index = oe[target_col].sort_values().index.values[-15]
ax.axvline(oe.loc[truth_index, target_col], color='r')
fig.tight_layout();

Lets keep that for later.

In [ ]:
truth = oe.loc[truth_index]

Drop the truth form the training data set

In [ ]:
data = oe.loc[~oe.index.isin([truth_index]), :]
assert data.shape[0] == oe.shape[0] - 1

## Set observation values and weights

Use the simulated values from the truth real as calibration targets.

In [ ]:
obs = pst.observation_data
obs.loc[truth.index,'obsval'] = truth.values
obs.oname.unique()

In [ ]:
obs.loc[target_col]

We have a little utility function in `herebdragons.py` to get the cell ids that correspond to "measured" heads (btw, these match obs locations in the original non-python tutorials):

In [ ]:
calib_obs = hbd.get_obs_cellids(org_md)
calib_obs.head()

Set non-zero weights to these observations. We are history matching for the historical (past) flow stress period. 

We also have measured heads at specified locations, as well as a section of the river that is gauged. Lets just use an arbitrary assumption of stdv of 0.1 and 0.0001 for heads and riv rate, respectively.

In [ ]:
_obs = obs.loc[obs.oname=='heads0'].copy()
_obs['i'] = _obs['i'].astype(int)
_obs.sort_values(by=['i'],inplace=True)

nzobsnmes = _obs.loc[_obs['i'].isin(calib_obs.icpl.values)].obsnme.tolist()
assert len(nzobsnmes) >0

obs.loc[nzobsnmes,'weight'] = 1.0 / 0.1
obs.loc[nzobsnmes,'standard_deviation'] = 0.1

obs.loc[obs.oname=='riv', 'weight'] = 1.0 / 0.0001
obs.loc[obs.oname=='riv', 'standard_deviation'] = 0.0001

Just for fun, lets look at the true K and hyperparameter fields:

In [ ]:
onames =['k', 'npfkpp-aniso', 'npfkpp-bearing', 'npfkpp-corrlen']

fig,axs = plt.subplots(1,4,figsize=(16,4))

for e,oname in enumerate(onames):
    ax = axs[e]
    ax.set_aspect("equal")
    pm = flopy.plot.PlotMapView(model=gwf, ax=ax)

    _obs = obs.loc[obs.oname==oname].copy()
    _obs["i"] = _obs["i"].astype(int)
    _obs.sort_values("i", inplace=True)
    obsnmes = _obs.obsnme.tolist()
    arr = obs.loc[obsnmes].obsval.values
    if oname=='k':
        arr = np.log10(arr)
        oname = "log10(k)"

    pa = pm.plot_array(arr)
    plt.colorbar(pa, ax=ax, shrink=0.5)

    ax.set_title(oname)
    ax.set_xticks([])
    ax.set_yticks([])


fig.tight_layout();
plt.show()
plt.close();

And the true max temperature field. This is our prediction of interest.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,5))

oname = "temp"

ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)

_obs = obs.loc[obs.oname==oname].copy()
_obs["i"] = _obs["i"].astype(int)
_obs.sort_values("i", inplace=True)
obsnmes = _obs.obsnme.tolist()
arr = obs.loc[obsnmes].obsval.values
#f oname=='k':
#    arr = np.log10(arr)

pa = pm.plot_array(arr)
plt.colorbar(pa, ax=ax, shrink=0.5)

ax.set_title(oname)
ax.set_xticks([])
ax.set_yticks([])


fig.tight_layout();
plt.show()
plt.close();

# Start DSI-AE

For DSI, we dont need all the observations we have been tracking. We only really need the non-zero weighted obs and the prediction sof interest. 

In [ ]:
obs = pst.observation_data
obs.oname.unique()

keep_obs = obs.loc[obs.weight > 0].obsnme.tolist()
keep_obs.extend(obs.loc[obs.oname=='temp'].obsnme.tolist())

Lets use normal score transformation (usualy a good choice..)

In [ ]:
transforms = [
            {"type":"normal_score"},
            #{"type":"standard_scaler"}
            ]

And now we are good to build the DSI-AE surrogate. Note we use the exact same arguments as we did in the DSI notebook...

In [ ]:
dsi = DSIAE(pst=pst, #optional...
          data = data[keep_obs],
          transforms=transforms,
           energy_threshold=.9999, # the truncated-svd energy threshold
          )

## Training the AE
Here is where things start to differ.  For `DSI`, "fitting" the model just meant doing SVD. For `DSIAE` it means training the neural net to encode/decode the data set. There is a bit more user input and options for hyperparameter tunning available.

### Machine learning dependency
The `DSIAE` class uses `tensorflow` in the background. `Tensorflow` is an open-source machine learning framework developed by Google, widely used for building and training neural networks, including autoencoders. It provides flexible tools for both research and production, supporting CPUs, GPUs, and TPUs for scalable computation. While powerful, `tensorflow`s performance and installation can vary across operating systems—GPU acceleration, for instance, often requires careful setup of compatible CUDA and cuDNN libraries on Windows or Linux. Additionally, version compatibility between `tensorflow`, Python, and hardware drivers can sometimes pose challenges in maintaining stable environments.

It is not our intention here to provide traning on machine learning (there are many btter resources out there to lear from...). We assume the reader has some knowledge on machine learning, and the use of tensorflow. Here are some basic insghts:

* **validation_split=0.1** – Reserves 10% of the data for validation, helping monitor overfitting during training. Typical values range from 0.1 to 0.2; larger splits may be used when data are abundant.
* **hidden_dims=(128, 64)** – Defines the number and size of hidden layers in the autoencoder. More layers or larger dimensions increase model capacity but may cause overfitting; smaller architectures are preferable for simpler data.
* **lr=0.0001** – The learning rate controls how quickly the model updates its weights. Too high can cause instability; too low can slow convergence. Start with 1e-3 or 1e-4 and adjust based on training behaviour.
* **epochs=300** – The number of complete passes through the dataset. More epochs allow better learning but risk overfitting; early stopping can mitigate this.
* **batch_size=32** – The number of samples processed before updating model weights. Smaller batches improve generalization but train slower; 32–128 is common.
* **early_stopping=True** – Stops training automatically if validation loss stops improving, preventing overfitting and saving time.



In [ ]:
dsi.fit(
        validation_split=0.1, # the fraction of data to use for validation during training
        hidden_dims=(64,32),  # the architecture of the autoencoder
        lr=0.001,  # learning rate for the optimizer
        epochs=300,  # number of training epochs
        batch_size=32,  # size of each training batch
        early_stopping=True,  # whether to stop training early if validation loss doesn't improve
        random_state=42,  # random seed for reproducibility
        );

If you wish, pyemu has a built in hyperparameetr tuner (again, just using tensorflow in the background) that undertakes a grid serach to do hyperparameter tunning. Uncomment the code below if you wish to exeriment. It can take a few minutes...

In [ ]:
#results = dsi.hyperparam_search(
#                latent_dims=None,
#                latent_dim_mults = [0.75, 1.0, 1.25],
#                hidden_dims_list=[(64,32),(128,64),(256,128)],
#                lrs=[1e-3, 1e-4, 1e-5],
#                epochs=300,
#                batch_size=32,
#                random_state=42
#            )

## Encoding/Decoding

A convenient aspect of DSI-AE versus vanilla DSI, is that we can directly project/encode the physics-based model outputs directly to latent space and back again. This allows us to estimate the "error" we incurr from simplification across data space. 

Here is how we do it:

In [ ]:
Z = dsi.encode(dsi.data)
Z

`Z` is the ensemble of latent space parameters that directly correspond to the original "prior" ensemble of FullOrderModel (FOM) outputs. These are the parameters that `pestpp` sees.

In [ ]:
Z

We can project these back to data space by "predciting" (aka decoding) with the dsiae object:

In [ ]:
X_hat = dsi.predict(Z)
X_hat

Let's see how well we did at reconstructing the `base` realization.

Pretty good, with some error (as expected).

In [ ]:
plt.scatter(data.loc['base',keep_obs],
            X_hat.loc['base',keep_obs],
            marker='.',
            alpha=0.5)
plt.xlabel('Original Data')
plt.ylabel('Reconstructed Data')
plt.title('Original vs Reconstructed Data Scatter Plot')
plt.show()

# Setup for pestpp

And setup the DSI pest dir...

key detail now, spceify the `use_runstor=True` argument to setup the forward run using the external manager:

In [ ]:
dsi_t_d = "pst_template_dsiae"

dpst = dsi.prepare_pestpp(t_d = dsi_t_d,
                          use_runstor=True)
dpst

We can take a look at what the forward run function looks like:

In [ ]:
#open forward_run.py and print
with open(os.path.join(dsi_t_d, 'forward_run.py'),'r') as f:
    print(f.read())

## AE latent space prior

In vanilla DSI, the latent space prior is straightforward: Guasiian normal distributions with mean 0 and stdv 1 for all latent space parameters.

In DSI-AE things get a bit more complicated. Autoencoder-based approaches learn a nonlinear latent space, and the resulting latent parameter distribution is generally non-Gaussian and data-driven. This flexibility allows autoencoders to capture more complex, multimodal relationships in the data, though it can make interpretation and sampling from the latent space less straightforward than in PCA-based methods.

Lets take a look at one of them:

In [ ]:
Z.iloc[:,0].hist()
plt.xlabel('Latent Parameter')
plt.ylabel('Frequency')
plt.title('Histogram of Latent Parameter')
plt.show()

If we are happy using the same number of realizations with DSIAE as we had realizations in the training data set, then we are fine. We can explicilty pass the encoded FOM prior values as the dsiae-prior parameter ensemble to pestpp-ies.

If we want to use a larger ensemble (usualy good ideia if we can aford it..), then we need to do a bit more work to empirically sample the dsi-ae latent parameter prior distribution...

Lets do that now, using some tricks built into pyemu. First, load the latent space prior as a `ParameterEnsemble` object. This was pre-preared when you called `.prepare_pespp()`

In [ ]:
dpst.pestpp_options['ies_parameter_ensemble']

In [ ]:
latent_pe = pyemu.ParameterEnsemble.from_binary(dpst,
                                                os.path.join(dsi_t_d, dpst.pestpp_options['ies_parameter_ensemble']))
latent_pe.tail()

In [ ]:
latent_pe.shape

We can now use the `draw_new_ensemble()` function to draw new realizations...

In [ ]:
nreals = 1000

In [ ]:
pe_ext = latent_pe.draw_new_ensemble(nreals-latent_pe.shape[0])
#pe_ext.enforce()
pe_ext.shape, latent_pe.shape

Lets comapre the distirbution of one of the parameters to see what this looks like:

In [ ]:
latent_pe.iloc[:,0].hist()
pe_ext.iloc[:,0].hist(alpha=0.5)
plt.xlabel('Latent Parameter')
plt.ylabel('Frequency')
plt.title('Histogram of Latent Parameter with Extended Ensemble')
plt.show()

Great! Merge them together:

In [ ]:
pe_ext = pd.concat([pe_ext,latent_pe],axis=0)
# find index position of 'base' in latent_pe
idx = np.where(pe_ext.index.values == 'base')[0][0]
pe_ext.reset_index(drop=True,inplace=True)
# name the row at idx 'base'
pe_ext.rename(index={idx:'base'},inplace=True)

assert (pe_ext.loc['base'] == latent_pe.loc['base']).all()

And write the extended parametr ensemble back to the pest dir, whislt also updateing the pespp options.

In [ ]:
pyemu.ParameterEnsemble(dpst,pe_ext).to_binary(os.path.join(dsi_t_d,'prior_extended.jcb'))
dpst.pestpp_options['ies_parameter_ensemble'] = "prior_extended.jcb"
pe_ext.tail()

And we are good to go...sheesh that was hard.

Lets  run for 3 iterations. WThis is more that we diod for DSI. Why? Becasue the relationship is not as linear as for DSI, getting a good fit is more chalenging.

In [ ]:
dpst.control_data.noptmax = 3
dpst.pestpp_options["ies_num_reals"] = nreals

In [ ]:
dpst.pestpp_options["ies_multimodal_alpha"] = 0.25

In [ ]:
dpst.write(os.path.join(dsi_t_d, "dsi.pst"),version=2)

Get the executables again...

In [ ]:
hbd.get_bins(dsi_t_d)

# Run pestpp with /e

Right on! We are ready to get cracking. Let's run pestpp-ies and see what we get. 

Make a copy for safekeeping...

In [ ]:
md = f"master_dsiae"

if os.path.exists(md):
    shutil.rmtree(md)
shutil.copytree(dsi_t_d, md)

Now we will run `pestpp-ies` with the external run manager option. We do this by calling `pestpp-ies [controlfile].pst /e`. The `/e` trigegrs the external run manager option.

Note that this run is in "serial" as far as `pestpp` is concerned. We are handling the paralelization of the dsi forward runs.

Here we go!

In [ ]:
pyemu.os_utils.run('pestpp-ies dsi.pst /e', cwd=md,verbose=True)

# Read pest results

In [ ]:
pst = pyemu.Pst(os.path.join(md,"dsi.pst"))

See how it take slonger to get better fits?

In [ ]:
pst.ies.phiactual

In [ ]:
obs = pst.observation_data
obs.oname.unique()

Get the posterior observation ensemble

In [ ]:
obsen = pst.ies.obsen.copy()
oe = obsen.loc[pst.control_data.noptmax]
oe.head()

See how well we fit the head obs data:

In [ ]:
nzobsnmes = pst.nnz_obs_names

nzobsnmes = pst.nnz_obs_names[:-1]
fig,axs = plt.subplots(1,2,figsize=(6,3))

ax = axs[0]
ax.set_aspect('equal')
ax.set_title('head obs')

[ax.scatter(obs.loc[nzobsnmes].obsval, oe.loc[i,nzobsnmes],c='b',marker='.') for i in oe.index];

xmax = max(ax.get_xlim()[0],ax.get_ylim()[0])
ymax = max(ax.get_xlim()[1],ax.get_ylim()[1])
limax = max(xmax,ymax)
xmin = min(ax.get_xlim()[0],ax.get_ylim()[0])
ymin = min(ax.get_xlim()[1],ax.get_ylim()[1])
limn = min(xmin,ymin)
ax.plot([limn,limax],[limn,limax],'r--')
ax.set_xlim(limn,limax)
ax.set_ylim(limn,limax)


nzobsnmes = pst.nnz_obs_names[-1:]
ax = axs[1]
ax.set_aspect('equal')
ax.set_title(nzobsnmes[0])

[ax.scatter(obs.loc[nzobsnmes].obsval, oe.loc[i,nzobsnmes],c='b',marker='.') for i in oe.index];

xmax = max(ax.get_xlim()[0],ax.get_ylim()[0])
ymax = max(ax.get_xlim()[1],ax.get_ylim()[1])
limax = max(xmax,ymax)
xmin = min(ax.get_xlim()[0],ax.get_ylim()[0])
ymin = min(ax.get_xlim()[1],ax.get_ylim()[1])
limn = min(xmin,ymin)
ax.plot([limn,limax],[limn,limax],'r--')
ax.set_xlim(limn,limax)
ax.set_ylim(limn,limax)

fig.tight_layout();

Make some plots of max temperature:

In [ ]:
oname = "temp"

for i in oe.index.values[-5:]:
    fig,ax = plt.subplots(1,1,figsize=(6,6))


    ax.set_aspect("equal")
    pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    #pm.plot_grid(alpha=0.1,lw=0.1)

    _obs = obs.loc[obs.oname==oname].copy()
    _obs["i"] = _obs["i"].astype(int)
    _obs.sort_values("i", inplace=True)
    obsnmes = _obs.obsnme.tolist()
    diverg = max(oe.loc[:,obsnmes].values.max(), abs(oe.loc[:,obsnmes].values.min()))
    vmax = np.ceil(diverg)
    vmin = 16 
    
    arr = oe.loc[i,obsnmes].values
    #arr = arr.reshape(61,gwf.modelgrid.ncpl).max(axis=0)
    #f oname=='k':
    #    arr = np.log10(arr)

    pa = pm.plot_array(arr, cmap="Reds", vmin=vmin,vmax=vmax)
    plt.colorbar(pa, ax=ax, shrink=0.5)

    ax.set_title(oname)
    ax.set_xticks([])
    ax.set_yticks([])

    fig.tight_layout();
    plt.show()
    plt.close();

Now lets compare to the truth and look at how data assimilation reduced predictive uncertainty:

In [ ]:

_obs = obs.loc[obs.oname==oname].copy()
_obs["i"] = _obs["i"].astype(int)
_obs.sort_values("i", inplace=True)
obsnmes = _obs.obsnme.tolist()


fig,axs = plt.subplots(2,2,figsize=(12,12))


ax = axs[0,0]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
#pm.plot_grid(alpha=0.1,lw=0.1)

arr = oe.loc[:,obsnmes].mean()


vmax = max(oe.loc[:,obsnmes].values.max(), abs(oe.loc[:,obsnmes].values.min()))
vmin = oe.loc[:,obsnmes].values.min()
vmax = np.ceil(vmax)
vmin = 16 
pa = pm.plot_array(arr, cmap="Reds", vmin=vmin, vmax=vmax)
plt.colorbar(pa, ax=ax, shrink=0.5)

ax.set_title('dsi(mean)')


ax = axs[0,1]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
#pm.plot_grid(alpha=0.1,lw=0.1)

arr = oe.loc[:,obsnmes].std()


vmax = max(oe.loc[:,obsnmes].values.max(), abs(oe.loc[:,obsnmes].values.min()))
vmin = oe.loc[:,obsnmes].values.min()
vmax = np.ceil(vmax)
vmin = 16 
pa = pm.plot_array(arr, cmap="Reds",)# vmin=vmin, vmax=vmax)
plt.colorbar(pa, ax=ax, shrink=0.5)


cell = int(obs.loc[target_col].i)
arr = arr.values
arr[:] = np.nan
arr[cell] = 1.0
pa = pm.plot_array(arr,)# vmin=vmin, vmax=vmax)

ax.set_title('dsi(std)')


ax = axs[1,0]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
#pm.plot_grid(alpha=0.1,lw=0.1)

arr = oe.loc[:,obsnmes].mean()
arr = arr - obs.loc[obsnmes,"obsval"].values

vmax = max(arr.max(), abs(arr.min()))

pa = pm.plot_array(arr, cmap="RdBu", vmin=-vmax,vmax=vmax)
plt.colorbar(pa, ax=ax, shrink=0.5)

cell = int(obs.loc[target_col].i)
arr = arr.values
arr[:] = np.nan
arr[cell] = 1.0
pa = pm.plot_array(arr,)# vmin=vmin, vmax=vmax)

ax.set_title('dsi(mean) - truth')


ax = axs[1,1]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)

arr =  (data.loc[:,obsnmes].std() - oe.loc[:,obsnmes].std())/data.loc[:,obsnmes].std()
arr[abs(data.loc[:,obsnmes].std())<1e-3] = 0

pa = pm.plot_array(arr, cmap="RdBu_r",vmax=1,vmin=-1)#,vmin=-vmax,vmax=vmax )
plt.colorbar(pa, ax=ax, shrink=0.5)

ax.set_title('rel std unc reduction [(prior-post)/(prior)]')

for ax in axs.flatten():
    ax.set_xticks([])
    ax.set_yticks([])

fig.tight_layout();
plt.show()
plt.close();

In [ ]:
obs = pst.observation_data
obsnmes = obs.loc[obs.oname=="temp"].obsnme.tolist()


fig,ax = plt.subplots(1,1,figsize=(5,5))

data.loc[:,target_col].hist(ax=ax, alpha=0.5,color='fuchsia',zorder=0)

obsen.loc[0].loc[:,target_col].hist(ax=ax,color='0.5')
oe.loc[:,target_col].hist(ax=ax,alpha=0.5,color='b')

ax.axvline(obs.loc[target_col].obsval, color='r')
fig.tight_layout();